这是用来debug的notebook，目前已经测试能够给模型喂数据并且跑出来的概率是合理的

In [1]:
import torch
import argparse
import pickle
from utils.dict import Dictionary
from utils.dataloader import myDataloader
from model.model import ATT_model

In [2]:
params = {
    'dict': './temp/dictionary.pickle',
    'traindata': './temp/train_vec.pickle',
    'validdata': './temp/valid_vec.pickle',
    'batch_size': 32,
    'embedding_size': 384,
    'gru_size': 256,
    'dropout': 0.1
}


In [3]:
# 加载字典
with open(params['dict'], 'rb') as f:
    dictionary = pickle.load(f)

# 加载数据
with open(params['traindata'], 'rb') as tr, open(params['validdata'], 'rb') as v:
    train_vec = pickle.load(tr)
    valid_vec = pickle.load(v)

batched_train_data = myDataloader(dictionary, train_vec, params['batch_size'])
batched_valid_data = myDataloader(dictionary, valid_vec, params['batch_size'])

In [4]:
# 模型实例化
model = ATT_model(vocab_size=dictionary.len, embed_dim=params['embedding_size'], hidden_dim=params['gru_size'], dropout_rate=params['dropout'], PAD=0)

In [5]:
(documents, doc_lens),(querys, query_lens), answers = batched_train_data[0]

In [6]:
documents.dtype, doc_lens.dtype, querys.dtype, query_lens.dtype

(torch.int64, torch.int64, torch.int64, torch.int64)

In [7]:
prob, a, b, ba = model(documents, doc_lens, querys, query_lens)

In [12]:
prob[2].sum()

tensor(1.0000, grad_fn=<SumBackward0>)